In [1]:
import numpy as np
import pandas as pd
import re
import datetime as dt

In [3]:
Attempts = pd.read_excel('uefa.com/Attempts.xlsx')
Defending = pd.read_excel('uefa.com/Defending.xlsx').drop(columns=['Country'], inplace=False, axis=1)
Distribution = pd.read_excel('uefa.com/Distribution.xlsx').drop(columns=['Country'], inplace=False, axis=1)
Goalkeeping = pd.read_excel('uefa.com/Goalkeeping.xlsx').drop(columns=['Country'], inplace=False, axis=1)
Goals = pd.read_excel('uefa.com/Goals.xlsx').drop(columns=['Country'], inplace=False, axis=1)
#TODO: get and clean other stats from uefa site
Attacking = pd.read_excel('uefa.com/Attacking.xlsx')
Disciplinary = pd.read_excel('uefa.com/Disciplinary.xlsx')
uefa_stats = Attempts.merge(Defending, on='Team').merge(Distribution,on='Team').merge(Goalkeeping, on='Team').merge(Goals,on='Team').merge(Attacking,on='Team').merge(Disciplinary,on='Team')

In [4]:

uefa_stats.to_csv('uefa.com/uefa_stats.csv',index=False)
uefa_stats.head()

,Team,Country_x,Total Attempts,Attempts On Target,Attempts Off Target,Blocked,Balls Recovered,Tackles,Tackles Won,Tackles lost,...,Attacks,Assists,Corners Taken,Offsides,Dribbles,Country,Fouls Commited,Fouls Suffered,Yellow Cards,Red Cards
0,Bayern München,Germany,184,70,59,55,336,123,55,68,...,520,12,69,16,158,Germany,78,64,14,1
1,Man City,England,149,55,43,51,264,75,31,44,...,587,10,64,8,137,England,61,56,10,0
2,Paris,France,149,49,57,43,352,111,46,65,...,524,13,57,11,191,France,75,86,10,1
3,PSV,Netherlands,141,57,47,37,331,144,53,91,...,391,12,49,13,105,Netherlands,83,69,20,1
4,Stuttgart,Germany,141,47,48,46,344,129,56,73,...,438,13,60,17,110,Germany,91,96,13,0


In [33]:
table_dict = {
    0:[],
    1:[],
    2:[],
    3:[],
    4:[],
    5:[],
    6:[],
    7:[]
}

with open('TeamStats.csv') as raw:
    i=0
    for line in raw:
        table_dict[i%8].append(int(line.strip('\n')))
        i += 1

table = pd.DataFrame.from_dict(table_dict)
table.rename(columns={0:'GP',1:'W',2:'D',3:'L',4:'F',5:'A',6:'GD',7:'P'}, inplace=True)

teams = pd.read_csv('Teams.csv')
table = teams.join(table)
table.to_csv('table.csv', index=False)

In [187]:
def extract_components(match_string):
        # Pattern explanation:
        # ([\d/]+):        - Captures the date (19/09)
        # \s*              - Any whitespace
        # (.+?)            - Captures team name (non-greedy)
        # \s+              - Whitespace
        # (\d+)-(\d+)      - Captures scores
        # \s+              - Whitespace
        # (.+)             - Captures away team name
        pattern = r'([\d]+)/([\d]+):\s*(.+?)\s+(\d+)-(\d+)\s+(.+)'

        match = re.match(pattern, match_string)
        if match:
            d = int(match.group(1))
            m = int(match.group(2))
            y = 2024 if m >=9 else 2025
            return (
                f'{d}/{m}/{y}',
                match.group(3).strip(),
                int(match.group(4)),
                int(match.group(5)),
                match.group(6).strip()
            )
        print(match_string)
        return None

In [192]:
results_with_dupes = pd.read_csv('other/results(withdupes).txt', header=None)
results = results_with_dupes.drop_duplicates(inplace=False)
results = results[0].apply(extract_components).tolist()
results = pd.DataFrame.from_records(results, columns=['date','home','HG','AG','away'])
results['date'] = pd.to_datetime(results['date'], format='%d/%m/%Y')
results.to_csv('other/results.csv', index=False)

In [193]:
results

,date,home,HG,AG,away
0,2024-09-19,Atalanta,0,0,Arsenal
1,2024-10-01,Arsenal,2,0,Paris Saint-Germain
2,2024-10-22,Arsenal,1,0,Shakhtar Donetsk
3,2024-11-06,Inter,1,0,Arsenal
4,2024-11-26,Sporting CP,1,5,Arsenal
...,...,...,...,...,...
143,2024-11-06,Shakhtar Donetsk,2,1,Young Boys
144,2025-01-21,Slovan Bratislava,1,3,Stuttgart
145,2024-10-01,Stuttgart,1,1,Sparta Praha
146,2024-10-22,Sturm Graz,0,2,Sporting CP


In [14]:
# players stats
team = 'bayern'
col_names=['bad','name','team-position','Minutes played', 'Matches played', 'Goals', 'Assists', 'Distance covered (km)', 'Top speed (km/h)']
keystats = pd.read_excel(f'uefa.com/players/{team}/keystats.xlsx',names=col_names).drop(0).drop(columns='bad')
col_names=['bad','name','team-position','Assists','Corners taken','Offsides','Dribbles','Matches played']
attack = pd.read_excel(f'uefa.com/players/{team}/attack.xlsx',names=col_names).drop(0).drop(columns=['bad','team-position','Matches played','Assists'])
col_names=['bad','name','team-position','Total attempts','Attempts on target','Attempts off target','Blocked','Matches played']
attempts = pd.read_excel(f'uefa.com/players/{team}/attempts.xlsx',names=col_names).drop(0).drop(columns=['bad','team-position','Matches played'])
col_names=['bad','name','team-position','Balls recovered','Tackles','Tackles won','Tackles lost','Clearances attempted','Matches played']
defend = pd.read_excel(f'uefa.com/players/{team}/defend.xlsx',names=col_names).drop(0).drop(columns=['bad','team-position','Matches played'])
col_names=['bad','name','team-position','Fouls committed','Fouls suffered','Yellow cards','Red cards','Minutes played','Matches played']
disciplinary = pd.read_excel(f'uefa.com/players/{team}/disciplinary.xlsx',names=col_names).drop(0).drop(columns=['bad','team-position','Matches played','Minutes played'])
col_names=['bad','name','team-position','Passing accuracy (%)','Passes attempted','Passes completed','Crossing accuracy (%)','Crosses attempted','Crosses completed','Free-kicks taken','Matches played']
distribution = pd.read_excel(f'uefa.com/players/{team}/distribution.xlsx',names=col_names).drop(0).drop(columns=['bad','team-position','Matches played'])
col_names=['bad','name','team-position','Saves','Goals conceded','Saves from penalties','Clean sheets','Punches made','Matches played']
goalkeeping = pd.read_excel(f'uefa.com/players/{team}/goalkeeping.xlsx',names=col_names).drop(0).drop(columns=['bad','team-position','Matches played'])
col_names=['bad','name','team-position','Goals','Right foot','Left foot','Head','Other','Goals inside area','Goals outside area','Penalties scored','Matches played']
goals = pd.read_excel(f'uefa.com/players/{team}/goals.xlsx',names=col_names).drop(0).drop(columns=['bad','team-position','Matches played','Goals'])
team_df = keystats.merge(attack, on='name').merge(attempts, on='name').merge(defend, on='name').merge(disciplinary, on='name').merge(distribution, on='name').merge(goalkeeping, on='name').merge(goals, on='name')
team_df[['team','position']] = team_df['team-position'].str.split(' - ', expand=True)
team_df.drop(columns='team-position', inplace=True)
# do i want to drop these?
team_df.replace(to_replace='-',value=np.nan,inplace=True)
team_df.to_csv(f'uefa.com/players/{team}/master.csv',index=False)


In [15]:
team_df

,name,Minutes played,Matches played,Goals,Assists,Distance covered (km),Top speed (km/h),Corners taken,Offsides,Dribbles,...,Punches made,Right foot,Left foot,Head,Other,Goals inside area,Goals outside area,Penalties scored,team,position
0,Kimmich,720,8,0,4,101.5,32.2,4,41,0,...,0,0,0,0,0,0,0,0,Bayern München,Midfielder
1,Kim,665,8,1,0,80.5,34.3,0,0,0,...,0,0,0,1,0,1,0,0,Bayern München,Defender
2,Olise,539,8,4,0,73.1,31.6,0,20,0,...,0,0,3,1,0,4,0,1,Bayern München,Midfielder
3,Musiala,510,8,2,4,63.5,34.1,4,2,2,...,0,1,0,1,0,2,0,0,Bayern München,Midfielder
4,Sané,382,8,1,0,46.9,34.4,0,3,0,...,0,0,1,0,0,0,1,0,Bayern München,Midfielder
5,Kane,630,7,6,1,78.5,30.4,1,0,4,...,0,5,0,1,0,6,0,3,Bayern München,Forward
6,Upamecano,630,7,0,0,77.8,34.3,0,0,0,...,0,0,0,0,0,0,0,0,Bayern München,Defender
7,Neuer,585,7,0,0,41.6,25.6,0,0,0,...,0,0,0,0,0,0,0,0,Bayern München,Goalkeeper
8,Gnabry,386,7,0,2,48.3,33.4,2,3,7,...,0,0,0,0,0,0,0,0,Bayern München,Forward
9,Müller,254,7,2,1,33.6,31.7,1,0,0,...,0,0,1,1,0,2,0,0,Bayern München,Forward


In [37]:
# combine all players into master players DF/file as well